In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm

In [2]:
champ_data = pd.read_excel("LeagueChampsData.xlsx")

In [3]:
champ_data['Champion'] = champ_data['Champion'].apply(lambda x: x.lower())

In [4]:
champ_data['Damage'] = champ_data["Damage"].apply(lambda x:1 if x=='Low' else (2 if x=="Med" else 3))
champ_data['Toughness'] = champ_data["Toughness"].apply(lambda x:1 if x=='Low' else (2 if x=="Med" else 3))
champ_data['CC'] = champ_data["CC"].apply(lambda x:1 if x=='Low' else (2 if x=="Med" else 3))
champ_data['Mobility'] = champ_data["Mobility"].apply(lambda x:1 if x=='Low' else (2 if x=="Med" else 3))
champ_data['DamageType'] = champ_data["DamageType"].apply(lambda x:1 if x=='AP' else (2 if x=="AD" else 3))

In [5]:
SUPPORT_ITEM_LIST = ["Bloodsong","Celestial Opposition","Dream Maker","Solstice Sleigh","Zaz'Zak's Realmspike"]
LEGENDARY_ITEM_LIST = ["Abyssal Mask","Archangel's Staff","Ardent Censer","Axiom Arc","Banshee's Veil","Black Cleaver",
                      "Blackfire Torch","Blade of the Ruined King","Bloodletter's Curse","Bloodthirster","Chempunk Chainsword",
                      "Cosmic Drive","Cryptbloom","Dawncore","Dead Man's Plate","Death's Dance","Echoes of Helia","Eclipse","Edge of Night",
                      "Essence Reaver","Experimental Hexplate","Fimbulwinter","Force of Nature","Frozen Heart","Guardian Angel","Guinsoo's Rageblade",
                      "Heartsteel","Hextech Rocketbelt","Hollow Radiance","Horizon Focus","Hubris","Hullbreaker","Iceborn Gauntlet",
                       "Immortal Shieldbow","Imperial Mandate","Infinity Edge","Jak'Sho, The Protean","Kaenic Rookern","Knight's Vow",
                      "Kraken Slayer","Liandry's Torment","Lich Bane","Locket of the Iron Solari","Lord Dominik's Regards","Luden's Companion",
                      "Malignance","Manamune","Maw of Malmortius","Mejai's Soulstealer","Mercurial Scimitar","Mikael's Blessing",
                      "Moonstone Renewer","Morellonomicon","Mortal Reminder","Muramana","Nashor's Tooth","Navori Flickerblade",
                      "Opportunity","Overlord's Bloodmail","Phantom Dancer","Profane Hydra","Rabadon's Deathcap","Randiun's Omen",
                      "Rapid Firecannon","Ravenous Hydra","Redemption","Riftmaker","Rod of Ages","Runaan's Hurricane","Rylai's Crystal Scepter",
                      "Seraph's Embrace","Serpent's Fang","Serylda's Grudge","Shadowflame","Shurelya's Battlesong","Spear of Shojin",
                      "Spirit's Visage","Staff of Flowing Water","Statikk Shiv","Sterak's Gage","Stormsurge","Stridebreaker",
                      "Sundered Sky","Sunfire Aegis","Terminus","The Collector","Thornmail","Titanic Hydra","Trailblazer","Trinity Force",
                      "Umbral Glaive","Unending Despair","Vigilant Wardstone","Void Staff","Voltaic Cyclosword","Warmog's Armor",
                      "Winter's Approach","Wit's End","Youmuu's Ghostblade","Yun Tal Wildarrows","Zeke's Convergence","Zhonya's Hourglass"]
BOOTS_LIST = ["Berserker's Greaves","Gunmetal Greaves","Mercury's Treads","Chainlaced Crushers",
              "Ionian Boots of Lucidity","Crimson Lucidity","Boots of Swiftness","Swiftmarch","Plated Steelcaps","Armored Advance",
              "Sorcerer's Shoes","Spellslinger's Shoes","Symbiotic Soles","Synchronized Souls","Forever Forward"]
SIMPLIFIED_BOOTS_LIST = ["Berserker's Greaves","Mercury's Treads","Ionian Boots of Lucidity",
                         "Boots of Swiftness","Plated Steelcaps","Sorcerer's Shoes","Symbiotic Soles"]

In [6]:
SUPPORT_ITEM_LIST = [item.lower() for item in SUPPORT_ITEM_LIST]
LEGENDARY_ITEM_LIST = [item.lower() for item in LEGENDARY_ITEM_LIST]
BOOTS_LIST = [item.lower() for item in BOOTS_LIST]
SIMPLIFIED_BOOTS_LIST = [item.lower() for item in SIMPLIFIED_BOOTS_LIST]

In [7]:
all_champs = champ_data["Champion"].tolist()

In [8]:
num_champs = len(all_champs)
num_items = len(LEGENDARY_ITEM_LIST)
num_boots = len(SIMPLIFIED_BOOTS_LIST)

In [9]:
champ_dict = {champ.lower():i for i,champ in enumerate(all_champs)}
item_dict = {item:i for i, item in enumerate(LEGENDARY_ITEM_LIST)}

In [10]:
match_data = pd.read_csv("league_matches.csv")

In [11]:
item_list = match_data["items"]

In [12]:
champs_list = match_data["enemy champs"].tolist()

### Convert items and champions from string to list

In [13]:
legendary_items = []
for match in item_list:
    match_items = []
    match = list(eval(match))
    for item in match:
        if item.lower() in LEGENDARY_ITEM_LIST:
            match_items.append(item.lower())
    legendary_items.append(match_items)

In [14]:
boots = []
for match in item_list:
    match_boots = []
    match = list(eval(match))
    for item in match:
        if item.lower() in BOOTS_LIST:
            item_to_add = item.lower()
            # Converting feats boots to regular boots
            if item_to_add == "gunmetal greaves":
                item_to_add = "berserker's greaves"
                
            if item_to_add == "chainlaced crushers":
                item_to_add = "mercury's treads"
                
            if item_to_add == "crimson lucidity":
                item_to_add = "ionian boots of lucidity"
                
            if item_to_add == "swiftmarch":
                item_to_add = "boots of swiftness"
                
            if item_to_add == "armored advance":
                item_to_add = "plated steelcaps"

            if item_to_add == "spellslinger's shoes":
                item_to_add = "sorcerer's shoes"

            if item_to_add == "forever forward" or item_to_add == "synchronized souls":
                item_to_add = "symbiotic soles"

            match_boots.append(item_to_add)
    boots.append(match_boots)

In [15]:
enemy_champs = []
enemy_champs_stats = []
for champ in champs_list:
    champions = list(eval(champ))
    enemy_champs.append([champion.lower() for champion in champions])

In [16]:
match_data['complete items'] = legendary_items

In [17]:
match_data['boots'] = boots

In [18]:
match_data['enemy champs'] = enemy_champs

### Make sure champs are Bot-Lane Champs (until all champs are added)

In [19]:
new_df = match_data[match_data["enemy champs"].apply(lambda lst: all(c in all_champs for c in lst))]

In [20]:
boots_data_df = new_df[new_df["boots"].apply(lambda lst: len(lst) > 0)]

In [21]:
boots_data_df

,kills,deaths,assists,cs,gold,vision,items,enemy champs,result,complete items,boots
1,4,7,15,53,12100,103,"['Solstice Sleigh', 'Titanic Hydra', 'Heartste...","[samira, lux]",0,"[titanic hydra, heartsteel, sterak's gage]",[mercury's treads]
2,0,13,20,37,10200,79,"['Oracle Lens', 'Solstice Sleigh', 'Heartsteel...","[twitch, lux]",0,"[heartsteel, warmog's armor]",[mercury's treads]
3,2,10,15,29,9800,79,"['Oracle Lens', 'Solstice Sleigh', 'Iceborn Ga...","[caitlyn, pyke]",1,"[iceborn gauntlet, warmog's armor]",[mercury's treads]
4,6,6,19,65,10500,59,"['Solstice Sleigh', 'Stridebreaker', ""Mercury'...","[jinx, sona]",1,"[stridebreaker, wit's end]",[mercury's treads]
5,2,7,4,31,6400,56,"['Solstice Sleigh', 'Heartsteel', ""Mercury's T...","[jinx, sona]",0,[heartsteel],[mercury's treads]
...,...,...,...,...,...,...,...,...,...,...,...
91,4,7,12,32,9600,69,"['Celestial Opposition', 'Heartsteel', 'Synchr...","[ashe, vel'koz]",1,"[heartsteel, sunfire aegis]",[symbiotic soles]
94,6,8,11,45,9600,64,"['Bloodsong', 'Eclipse', 'Fimbulwinter', 'Orac...","[varus, pyke]",0,"[eclipse, fimbulwinter]",[plated steelcaps]
95,1,7,19,25,8200,50,"['Celestial Opposition', 'Thornmail', ""Mercury...","[jinx, vel'koz]",1,[thornmail],[mercury's treads]
96,3,8,17,29,10000,77,"['Celestial Opposition', 'Heartsteel', 'Plated...","[vayne, brand]",1,"[heartsteel, winter's approach]",[plated steelcaps]


In [22]:
updated_champs_list = new_df["enemy champs"].tolist()
updated_champs_list_for_boots = boots_data_df["enemy champs"].tolist()

In [23]:
enemy_champs_dmg = []
enemy_champs_t = []
enemy_champs_cc = []
enemy_champs_mob = []
for champs in updated_champs_list:
    adc = champ_data[champ_data['Champion'] == champs[0]]
    sup = champ_data[champ_data['Champion'] == champs[1]]
    damage = adc['Damage'].values[0] + sup['Damage'].values[0]
    toughness = adc['Toughness'].values[0] + sup['Toughness'].values[0]
    cc = adc['CC'].values[0] + sup['CC'].values[0]
    mobility = adc['Mobility'].values[0] + sup['Mobility'].values[0]
    enemy_champs_dmg.append(damage)
    enemy_champs_t.append(toughness)
    enemy_champs_cc.append(cc)
    enemy_champs_mob.append(mobility)

In [24]:
enemy_champs_dmg_boots = []
enemy_champs_t_boots = []
enemy_champs_cc_boots = []
enemy_champs_mob_boots = []
for champs in updated_champs_list_for_boots:
    adc = champ_data[champ_data['Champion'] == champs[0]]
    sup = champ_data[champ_data['Champion'] == champs[1]]
    damage = adc['Damage'].values[0] + sup['Damage'].values[0]
    toughness = adc['Toughness'].values[0] + sup['Toughness'].values[0]
    cc = adc['CC'].values[0] + sup['CC'].values[0]
    mobility = adc['Mobility'].values[0] + sup['Mobility'].values[0]
    enemy_champs_dmg_boots.append(damage)
    enemy_champs_t_boots.append(toughness)
    enemy_champs_cc_boots.append(cc)
    enemy_champs_mob_boots.append(mobility)

### Convert champs and items to IDs

In [25]:
boots_df = pd.DataFrame()
boots_df['boots'] = boots_data_df['boots']
boots_df['enemy champs'] = boots_data_df['enemy champs']
boots_df['enemy champ ids'] = boots_df["enemy champs"].apply(lambda lst: [champ_dict[lst[0].lower()],champ_dict[lst[1].lower()]])
# Stats
boots_df['Damage'] = enemy_champs_dmg_boots
boots_df['Toughness'] = enemy_champs_t_boots
boots_df['CC'] = enemy_champs_cc_boots
boots_df['Mobility'] = enemy_champs_mob_boots
# Boots
boots_df['boots ids'] = boots_df["boots"].apply(lambda lst: [1 if item in lst else 0 for item in SIMPLIFIED_BOOTS_LIST])

In [26]:
boots_id_df = boots_df[['Damage','Toughness','CC','Mobility','boots ids']]

In [27]:
import torch
import torch.nn as nn
import torch.optim as optim

In [28]:
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')

In [29]:
class BootsModel(nn.Module):
    def __init__(self, num_champs, embed_dim, hidden_dim, output):
        super(BootsModel, self).__init__()
        self.embed = nn.Embedding(num_champs, embed_dim)
        
        self.fc1 = nn.Linear(embed_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output)

    def forward(self, enemy_champs):
        champion_embeddings = self.embed(enemy_champs)
        x = champion_embeddings.mean(dim=1)
        x = self.fc1(x)
        x = self.fc2(x)
        return torch.sigmoid(x)

In [30]:
x_boots = [[]] * boots_id_df.shape[0]
dmg_b = boots_id_df['Damage'].tolist()
tuff_b = boots_id_df['Toughness'].tolist()
cc_b = boots_id_df['CC'].tolist()
mob_b = boots_id_df['Mobility'].tolist()

In [31]:
for i in range(boots_id_df.shape[0]):
    stats = [dmg_b[i],tuff_b[i],cc_b[i],mob_b[i]]
    x_boots[i] = x_boots[i] + stats

In [32]:
X_boots = torch.tensor(x_boots, dtype=torch.long).to(device)
y_boots = torch.tensor(boots_id_df['boots ids'].tolist()).to(device).float()

In [33]:
boots_embed_dim = 32
boots_hidden_dim = 32
embed_dim = 64
hidden_dim = 64

In [34]:
boots_model = BootsModel(num_champs, boots_embed_dim, boots_hidden_dim, num_boots)
boots_model.to(device)

BootsModel(
  (embed): Embedding(55, 32)
  (fc1): Linear(in_features=32, out_features=32, bias=True)
  (fc2): Linear(in_features=32, out_features=7, bias=True)
)

In [35]:
learning_rate = 1e-4
optimizer_boots = optim.Adam(boots_model.parameters(), lr=learning_rate)
criterion_boots = nn.BCELoss()
epochs = 5000

In [36]:
for epoch in tqdm(range(epochs)):
    boots_model.train()
    optimizer_boots.zero_grad()
    outputs = boots_model(X_boots)
    loss = criterion_boots(outputs, y_boots)
    loss.backward()
    optimizer_boots.step()

100%|█████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:05<00:00, 970.41it/s]


### Test Boots Model

In [37]:
enemy_adc = input("Enemy ADC: ").lower()
enemy_sup = input("Enemy Support: ").lower()
adc = champ_data[champ_data['Champion'] == enemy_adc]
sup = champ_data[champ_data['Champion'] == enemy_sup]
enemy_adc = champ_dict[enemy_adc]
enemy_sup = champ_dict[enemy_sup]
damage = adc['Damage'].values[0] + sup['Damage'].values[0]
toughness = adc['Toughness'].values[0] + sup['Toughness'].values[0]
cc = adc['CC'].values[0] + sup['CC'].values[0]
mobility = adc['Mobility'].values[0] + sup['Mobility'].values[0]

Enemy ADC:  samira
Enemy Support:  lux


In [38]:
boots_model.eval()
with torch.no_grad():
    enemy_input = torch.tensor([[damage, toughness, cc, mobility]], dtype=torch.long).to(device)
    
    preds = boots_model(enemy_input)

In [39]:
probs = preds.tolist()[0]
item_prob = {}
for i in range(num_boots):
    item_prob[SIMPLIFIED_BOOTS_LIST[i]] = probs[i]

In [40]:
item_prob

{"berserker's greaves": 0.00040013325633481145,
 "mercury's treads": 0.32258644700050354,
 'ionian boots of lucidity': 0.01710861176252365,
 'boots of swiftness': 0.4144916832447052,
 'plated steelcaps': 0.17084954679012299,
 "sorcerer's shoes": 0.0018632280407473445,
 'symbiotic soles': 0.016516096889972687}

In [41]:
values = [v.item() if isinstance(v, torch.Tensor) else v for v in item_prob.values()]
max_index = values.index(max(values))

In [42]:
one_hot = [1 if i == max_index else 0 for i in range(len(values))]

In [43]:
sorted_item_probs = sorted(item_prob.items(), key = lambda item: item[1], reverse = True)

In [44]:
sorted_item_probs

[('boots of swiftness', 0.4144916832447052),
 ("mercury's treads", 0.32258644700050354),
 ('plated steelcaps', 0.17084954679012299),
 ('ionian boots of lucidity', 0.01710861176252365),
 ('symbiotic soles', 0.016516096889972687),
 ("sorcerer's shoes", 0.0018632280407473445),
 ("berserker's greaves", 0.00040013325633481145)]

In [45]:
predicted_boots = sorted_item_probs[0][0]

In [46]:
predicted_boots

'boots of swiftness'

### Train Final Model

In [47]:
final_df = pd.DataFrame()
final_df['complete items'] = new_df['complete items']
final_df['enemy champs'] = new_df['enemy champs']
final_df['enemy champ ids'] = final_df["enemy champs"].apply(lambda lst: [champ_dict[lst[0].lower()],champ_dict[lst[1].lower()]])
# Stats
final_df['Damage'] = enemy_champs_dmg
final_df['Toughness'] = enemy_champs_t
final_df['CC'] = enemy_champs_cc
final_df['Mobility'] = enemy_champs_mob
# Final items
final_df['item ids'] = final_df["complete items"].apply(lambda lst: [1 if item in lst else 0 for item in LEGENDARY_ITEM_LIST])

In [48]:
final_df

,complete items,enemy champs,enemy champ ids,Damage,Toughness,CC,Mobility,item ids
1,"[titanic hydra, heartsteel, sterak's gage]","[samira, lux]","[13, 33]",6,3,3,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,"[heartsteel, warmog's armor]","[twitch, lux]","[18, 33]",6,2,3,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,"[iceborn gauntlet, warmog's armor]","[caitlyn, pyke]","[2, 41]",5,2,5,5,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,"[stridebreaker, wit's end]","[jinx, sona]","[6, 46]",5,2,4,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,[heartsteel],"[jinx, sona]","[6, 46]",5,2,4,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...,...
93,[heartsteel],"[ashe, lux]","[1, 33]",5,2,5,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
94,"[eclipse, fimbulwinter]","[varus, pyke]","[19, 41]",5,2,5,4,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
95,[thornmail],"[jinx, vel'koz]","[6, 51]",6,2,4,2,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
96,"[heartsteel, winter's approach]","[vayne, brand]","[20, 27]",6,2,4,3,"[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [49]:
id_df = final_df[['Damage','Toughness','CC','Mobility']]

In [50]:
id_df

,Damage,Toughness,CC,Mobility
1,6,3,3,4
2,6,2,3,3
3,5,2,5,5
4,5,2,4,2
5,5,2,4,2
...,...,...,...,...
93,5,2,5,2
94,5,2,5,4
95,6,2,4,2
96,6,2,4,3


In [51]:
boot_keys = [
    "berserker's greaves",
    "mercury's treads",
    'ionian boots of lucidity',
    'boots of swiftness',
    'plated steelcaps',
    "sorcerer's shoes",
    'symbiotic soles'
]

In [52]:
predicted_boots = []

for _, row in id_df.iterrows():
    x = torch.tensor(row.values, dtype=torch.long).unsqueeze(0).to(device)
    
    # model prediction
    with torch.no_grad():
        output = boots_model(x)
        probs = torch.softmax(output, dim=1).squeeze()

    # convert to one-hot
    max_index = torch.argmax(probs).item()
    one_hot = [1 if i == max_index else 0 for i in range(len(boot_keys))]
    
    predicted_boots.append(one_hot)
    
id_df['predicted boots'] = predicted_boots

C:\Users\HP\AppData\Local\Temp\ipykernel_33116\2387440308.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_df['predicted boots'] = predicted_boots


In [53]:
id_df['item ids'] = final_df['item ids']

C:\Users\HP\AppData\Local\Temp\ipykernel_33116\1451306600.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  id_df['item ids'] = final_df['item ids']


In [54]:
id_df

,Damage,Toughness,CC,Mobility,predicted boots,item ids
1,6,3,3,4,"[0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,6,2,3,3,"[0, 0, 0, 1, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
3,5,2,5,5,"[0, 0, 0, 0, 0, 0, 1]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
4,5,2,4,2,"[0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
5,5,2,4,2,"[0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...
93,5,2,5,2,"[0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
94,5,2,5,4,"[0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
95,6,2,4,2,"[0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
96,6,2,4,3,"[0, 1, 0, 0, 0, 0, 0]","[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."


In [55]:
class BuildReccomender(nn.Module):
    def __init__(self, num_champs, embed_dim, hidden_dim, output):
        super(BuildReccomender, self).__init__()
        self.embed = nn.Embedding(num_champs, embed_dim)
        
        self.fc1 = nn.Linear(embed_dim, hidden_dim)
        self.fc2 = nn.Linear(hidden_dim, output)

    def forward(self, enemy_champs):
        champion_embeddings = self.embed(enemy_champs)
        x = champion_embeddings.mean(dim=1)
        x = self.fc1(x)
        x = self.fc2(x)
        return torch.sigmoid(x)

In [56]:
x = [[]] * id_df.shape[0]
dmg = id_df['Damage'].tolist()
tuff = id_df['Toughness'].tolist()
cc = id_df['CC'].tolist()
mob = id_df['Mobility'].tolist()
boots = id_df['predicted boots'].tolist()

In [57]:
for i in range(id_df.shape[0]):
    stats = [dmg[i],tuff[i],cc[i],mob[i]]
    for boots_encoded in boots[i]:
        stats.append(boots_encoded)
    x[i] = x[i] + stats

In [58]:
X = torch.tensor(x, dtype=torch.long).to(device)
y = torch.tensor(id_df['item ids'].tolist()).to(device).float()

In [59]:
model = BuildReccomender(num_champs, embed_dim, hidden_dim, num_items)
model.to(device)

BuildReccomender(
  (embed): Embedding(55, 64)
  (fc1): Linear(in_features=64, out_features=64, bias=True)
  (fc2): Linear(in_features=64, out_features=102, bias=True)
)

In [60]:
learning_rate = 1e-4
optimizer = optim.Adam(model.parameters(), lr=learning_rate)
criterion = nn.BCELoss()
epochs = 5000

In [61]:
epochs = 5000
for epoch in tqdm(range(epochs)):
    model.train()
    optimizer.zero_grad()
    outputs = model(X)
    loss = criterion(outputs, y)
    loss.backward()
    optimizer.step()

100%|████████████████████████████████████████████████████████████████████████████| 5000/5000 [00:04<00:00, 1011.17it/s]


### Test model

In [62]:
enemy_adc = input("Enemy ADC: ").lower()
enemy_sup = input("Enemy Support: ").lower()
adc = champ_data[champ_data['Champion'] == enemy_adc]
sup = champ_data[champ_data['Champion'] == enemy_sup]
enemy_adc = champ_dict[enemy_adc]
enemy_sup = champ_dict[enemy_sup]
damage = adc['Damage'].values[0] + sup['Damage'].values[0]
toughness = adc['Toughness'].values[0] + sup['Toughness'].values[0]
cc = adc['CC'].values[0] + sup['CC'].values[0]
mobility = adc['Mobility'].values[0] + sup['Mobility'].values[0]

input_values = [damage, toughness, cc, mobility]
with torch.no_grad():
    enemy_input = torch.tensor([input_values], dtype=torch.long).to(device)
    preds_boots = boots_model(enemy_input)

    # convert to one-hot
    max_index = torch.argmax(preds_boots).item()
    one_hot = [1 if i == max_index else 0 for i in range(len(boot_keys))]

    for boots_encoded in one_hot:
        input_values.append(boots_encoded)

    enemy_input = torch.tensor([input_values], dtype=torch.long).to(device)
    preds = model(enemy_input)

Enemy ADC:  samira
Enemy Support:  lux


In [63]:
probs = preds.tolist()[0]
item_prob = {}
for i in range(num_items):
    item_prob[LEGENDARY_ITEM_LIST[i]] = probs[i]

In [64]:
sorted_item_probs = sorted(item_prob.items(), key = lambda item: item[1], reverse = True)

In [65]:
sorted_item_probs[:5]

[('iceborn gauntlet', 0.4171794354915619),
 ('heartsteel', 0.4126244783401489),
 ('locket of the iron solari', 0.12435603141784668),
 ("warmog's armor", 0.12215151637792587),
 ("shurelya's battlesong", 0.10989121347665787)]

### Save model

In [66]:
torch.save(boots_model.state_dict(),"boots_model.pth")

In [67]:
torch.save(model.state_dict(),"model_withboots.pth")